<a href="https://colab.research.google.com/github/multimodalart/majesty-diffusion/blob/main/latent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Latent Majesty Diffusion v1.3
#### Formerly known as Princess Generator
##### Access our [Majestic Guide](https://multimodal.art/majesty-diffusion) (_under construction_), our [GitHub](https://github.com/multimodalart/majesty-diffusion), join our community on [Discord](https://discord.gg/yNBtQBEDfZ) or reach out via [@multimodalart on Twitter](https://twitter.com/multimodalart))
\
 
---
\


#### CLIP Guided Latent Diffusion by [dango233](https://github.com/Dango233/) and [apolinario (@multimodalart)](https://twitter.com/multimodalart). 
The LAION-400M-trained model and the modified inference code are from [CompVis Latent Diffusion](https://github.com/CompVis/latent-diffusion). The guided-diffusion method is modified by Dango233 based on [Katherine Crowson](https://twitter.com/RiversHaveWings)'s guided diffusion notebook. multimodalart savable settings, MMC and assembled the Colab. Check the complete list on our GitHub. Some functions and methods are from various code masters (nsheppard, DanielRussRuss and others)

Changelog: 1.3 - better upscaler (learn how to use it on our [Majestic Guide](https://multimodal.art/majesty-diffusion))

## Save model and outputs on Google Drive? 

In [ ]:
#@markdown Enable saving outputs to Google Drive to save your creations at AI/models
save_outputs_to_google_drive = True #@param {type:"boolean"}
#@markdown Enable saving models to Google Drive to avoid downloading the model every Colab instance
save_models_to_google_drive = True #@param {type:"boolean"}

if save_outputs_to_google_drive or save_models_to_google_drive:
    from google.colab import drive
    try:
      drive.mount('/content/gdrive')
    except:
      save_outputs_to_google_drive = False
      save_models_to_google_drive = False

model_path = "/content/gdrive/MyDrive/AI/models" if save_models_to_google_drive else "/content/"
outputs_path = "/content/gdrive/MyDrive/AI/latent_majesty_diffusion" if save_outputs_to_google_drive else "/content/outputs"
!mkdir -p $model_path
!mkdir -p $outputs_path
print(f"Model will be stored at {model_path}")
print(f"Outputs will be saved to {outputs_path}")

#If you want to run it locally change it to true
is_local = False
skip_installs = False
if(is_local):
  model_path = "/choose/your/local/model/path"
  outputs_path = "/choose/your/local/outputs/path"
  skip_installs = True

In [ ]:
#@title Model settings
#@markdown The `original` model is the model trained by CompVis in the LAION-400M dataset
#@markdown <br>The `finetuned` model is a finetune of the `original` model by Jack000 that generates less watermarks, but is a bit worse in text synthesis. Colab Free does not have enough run for the finetuned (for now)
latent_diffusion_model = 'original' #@param ["original", "finetuned"]

# Setup stuff

In [ ]:
#@title Installation
if(not skip_installs):
    import subprocess
    nvidiasmi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    cards_requiring_downgrade = ["Tesla T4", "V100"]
    if any(cardstr in nvidiasmi_output for cardstr in cards_requiring_downgrade):
        downgrade_pytorch_result = subprocess.run(['pip', 'install', 'torch==1.10.2', 'torchvision==0.11.3', '-q'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    import sys
    sys.path.append(".")
    !git clone https://github.com/multimodalart/latent-diffusion
    !git clone https://github.com/CompVis/taming-transformers
    !git clone https://github.com/TencentARC/GFPGAN
    !git clone https://github.com/NightmareAI/majesty-diffusion
    !git lfs clone https://github.com/LAION-AI/aesthetic-predictor
    !pip install -e ./taming-transformers
    !pip install omegaconf>=2.0.0 pytorch-lightning>=1.0.8 torch-fidelity einops
    !pip install transformers
    !pip install dotmap
    !pip install resize-right
    !pip install piq
    !pip install lpips
    !pip install basicsr
    !pip install facexlib
    !pip install realesrgan

    sys.path.append('./taming-transformers')
    from taming.models import vqgan
    from subprocess import Popen, PIPE
    try:
        import mmc
    except:
        # install mmc
        !git clone https://github.com/apolinario/Multi-Modal-Comparators --branch gradient_checkpointing
        !pip install poetry
        !cd Multi-Modal-Comparators; poetry build
        !cd Multi-Modal-Comparators; pip install dist/mmc*.whl
        
        # optional final step:
        #poe napm_installs
        !python Multi-Modal-Comparators/src/mmc/napm_installs/__init__.py
    # suppress mmc warmup outputs
    import mmc.loaders

Now, download the checkpoint (~5.7 GB). This will usually take 3-6 minutes.

In [ ]:
#@title Download models
import os
if os.path.isfile(f"{model_path}/latent_diffusion_txt2img_f8_large.ckpt"):
    print("Using Latent Diffusion model saved from Google Drive")
else:    
    !wget -O $model_path/latent_diffusion_txt2img_f8_large.ckpt https://ommer-lab.com/files/latent-diffusion/nitro/txt2img-f8-large/model.ckpt --no-check-certificate

if os.path.isfile(f"{model_path}/finetuned_state_dict.pt"):
    print("Using Latent Diffusion model saved from Google Drive")
else:    
    !wget -O $model_path/finetuned_state_dict.pt https://huggingface.co/multimodalart/compvis-latent-diffusion-text2img-large/resolve/main/finetuned_state_dict.pt --no-check-certificate

if os.path.isfile(f"{model_path}/ava_vit_l_14_336_linear.pth"):
  print("Using ViT-L/14@336px aesthetic model from Google Drive")
else:
  !wget -O $model_path/ava_vit_l_14_336_linear.pth https://multimodal.art/models/ava_vit_l_14_336_linear.pth

if os.path.isfile(f"{model_path}/sa_0_4_vit_l_14_linear.pth"):
  print("Using ViT-L/14 aesthetic model from Google Drive")
else:
  !wget -O $model_path/sa_0_4_vit_l_14_linear.pth https://multimodal.art/models/sa_0_4_vit_l_14_linear.pth

if os.path.isfile(f"{model_path}/ava_vit_l_14_linear.pth"):
  print("Using ViT-L/14 aesthetic model from Google Drive")
else:
  !wget -O $model_path/ava_vit_l_14_linear.pth https://multimodal.art/models/ava_vit_l_14_linear.pth

if os.path.isfile(f"{model_path}/ava_vit_b_16_linear.pth"):
  print("Using ViT-B/16 aesthetic model from Google Drive")
else:
  !wget -O $model_path/ava_vit_b_16_linear.pth http://batbot.tv/ai/models/v-diffusion/ava_vit_b_16_linear.pth
if os.path.isfile(f"{model_path}/sa_0_4_vit_b_16_linear.pth"):
  print("Using ViT-B/16 sa aesthetic model already saved")
else:
  !wget -O $model_path/sa_0_4_vit_b_32_linear.pth https://multimodal.art/models/sa_0_4_vit_b_16_linear.pth
if os.path.isfile(f"{model_path}/sa_0_4_vit_b_32_linear.pth"):
  print("Using ViT-B/32 aesthetic model from Google Drive")
else:
  !wget -O $model_path/sa_0_4_vit_b_32_linear.pth https://multimodal.art/models/sa_0_4_vit_b_32_linear.pth
if os.path.isfile(f"{model_path}/openimages_512x_png_embed224.npz"):
  print("Using openimages png from Google Drive")
else:
  !wget -O $model_path/openimages_512x_png_embed224.npz https://github.com/nshepperd/jax-guided-diffusion/raw/8437b4d390fcc6b57b89cedcbaf1629993c09d03/data/openimages_512x_png_embed224.npz
if os.path.isfile(f"{model_path}/imagenet_512x_jpg_embed224.npz"):
  print("Using imagenet antijpeg from Google Drive")
else:
  !wget -O $model_path/imagenet_512x_jpg_embed224.npz https://github.com/nshepperd/jax-guided-diffusion/raw/8437b4d390fcc6b57b89cedcbaf1629993c09d03/data/imagenet_512x_jpg_embed224.npz
if os.path.isfile(f"{model_path}/GFPGANv1.3.pth"):
  print("Using GFPGAN v1.3 from Google Drive")
else:
  !wget -O $model_path/GFPGANv1.3.pth https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth
!cp $model_path/GFPGANv1.3.pth GFPGAN/experiments/pretrained_models/GFPGANv1.3.pth


Let's also check what type of GPU we've got.

In [ ]:
!nvidia-smi

In [ ]:
#@title Import stuff
sys.path.append('./majesty-diffusion')
import majesty
import argparse, os, sys, glob
import torch
import numpy as np
from omegaconf import OmegaConf
from PIL import Image
from tqdm.auto import tqdm, trange
tqdm_auto_model = __import__("tqdm.auto", fromlist=[None]) 
sys.modules['tqdm'] = tqdm_auto_model
from einops import rearrange
from torchvision.utils import make_grid
import transformers
import gc
sys.path.append('./latent-diffusion')
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler
import tensorflow as tf
from dotmap import DotMap
import ipywidgets as widgets
from math import pi

from subprocess import Popen, PIPE

from dataclasses import dataclass
from functools import partial
import gc
import io
import math
import sys
import random
from piq import brisque
from itertools import product
from IPython import display
import lpips
from PIL import Image, ImageOps
import requests
import torch
from torch import nn
from torch.nn import functional as F
from torchvision import models
from torchvision import transforms
from torchvision import transforms as T
from torchvision.transforms import functional as TF
from numpy import nan
from threading import Thread
import time

#sys.path.append('../CLIP')
#Resizeright for better gradient when resizing
#sys.path.append('../ResizeRight/')
#sys.path.append('../cloob-training/')

from resize_right import resize

import clip
#from cloob_training import model_pt, pretrained

#pretrained.list_configs()
from torch.utils.tensorboard import SummaryWriter

In [ ]:
#@title Load the model
majesty.model_path = model_path
majesty.outputs_path = outputs_path
torch.backends.cudnn.benchmark = True
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
majesty.device = device

latent_diffusion_model = "finetuned"
config = OmegaConf.load(
    "./latent-diffusion/configs/latent-diffusion/txt2img-1p4B-eval.yaml"
)  # TODO: Optionally download from same location as ckpt and chnage this logic
model = majesty.load_model_from_config(
    config,
    f"{model_path}/latent_diffusion_txt2img_f8_large.ckpt",
    False,
    latent_diffusion_model,
)  # TODO: check path
model = model.half().eval().to(device)
# if(latent_diffusion_model == "finetuned"):
#  model.model = model.model.half().eval().to(device)

majesty.load_lpips_model()
# Alstro's aesthetic model
majesty.load_aesthetic_model()


# Run!

#### Perceptors (Choose your CLIP and CLIP-like models) 
Be careful if you don't pay for Colab Pro selecting more CLIPs might make you go out of memory. If you do have Pro, try adding ViT-L14 to your mix

In [ ]:
#@title Choose your perceptor models

# suppress mmc warmup outputs
import mmc.loaders
clip_load_list = []
#@markdown #### Open AI CLIP models
ViT_B32 = False #@param {type:"boolean"}
ViT_B16 = True #@param {type:"boolean"}
ViT_L14 = False #@param {type:"boolean"}
ViT_L14_336px = False #@param {type:"boolean"}
#RN101 = False #@param {type:"boolean"}
#RN50 = False #@param {type:"boolean"}
RN50x4 = False #@param {type:"boolean"}
RN50x16 = False #@param {type:"boolean"}
RN50x64 = False #@param {type:"boolean"}

#@markdown #### OpenCLIP models
ViT_B16_plus = False #@param {type: "boolean"}
ViT_B32_laion2b = True #@param {type: "boolean"}

#@markdown #### Multilangual CLIP models 
clip_farsi = False #@param {type: "boolean"}
clip_korean = False #@param {type: "boolean"}

#@markdown #### CLOOB models
cloob_ViT_B16 = False #@param {type: "boolean"}

# @markdown Load even more CLIP and CLIP-like models (from [Multi-Modal-Comparators](https://github.com/dmarx/Multi-Modal-Comparators))
model1 = "" # @param ["[clip - openai - RN50]","[clip - openai - RN101]","[clip - mlfoundations - RN50--yfcc15m]","[clip - mlfoundations - RN50--cc12m]","[clip - mlfoundations - RN50-quickgelu--yfcc15m]","[clip - mlfoundations - RN50-quickgelu--cc12m]","[clip - mlfoundations - RN101--yfcc15m]","[clip - mlfoundations - RN101-quickgelu--yfcc15m]","[clip - mlfoundations - ViT-B-32--laion400m_e31]","[clip - mlfoundations - ViT-B-32--laion400m_e32]","[clip - mlfoundations - ViT-B-32--laion400m_avg]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_e31]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_e32]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_avg]","[clip - mlfoundations - ViT-B-16--laion400m_e31]","[clip - mlfoundations - ViT-B-16--laion400m_e32]","[clip - sbert - ViT-B-32-multilingual-v1]","[clip - facebookresearch - clip_small_25ep]","[simclr - facebookresearch - simclr_small_25ep]","[slip - facebookresearch - slip_small_25ep]","[slip - facebookresearch - slip_small_50ep]","[slip - facebookresearch - slip_small_100ep]","[clip - facebookresearch - clip_base_25ep]","[simclr - facebookresearch - simclr_base_25ep]","[slip - facebookresearch - slip_base_25ep]","[slip - facebookresearch - slip_base_50ep]","[slip - facebookresearch - slip_base_100ep]","[clip - facebookresearch - clip_large_25ep]","[simclr - facebookresearch - simclr_large_25ep]","[slip - facebookresearch - slip_large_25ep]","[slip - facebookresearch - slip_large_50ep]","[slip - facebookresearch - slip_large_100ep]","[clip - facebookresearch - clip_base_cc3m_40ep]","[slip - facebookresearch - slip_base_cc3m_40ep]","[slip - facebookresearch - slip_base_cc12m_35ep]","[clip - facebookresearch - clip_base_cc12m_35ep]"] {allow-input: true}
model2 = "" # @param ["[clip - openai - RN50]","[clip - openai - RN101]","[clip - mlfoundations - RN50--yfcc15m]","[clip - mlfoundations - RN50--cc12m]","[clip - mlfoundations - RN50-quickgelu--yfcc15m]","[clip - mlfoundations - RN50-quickgelu--cc12m]","[clip - mlfoundations - RN101--yfcc15m]","[clip - mlfoundations - RN101-quickgelu--yfcc15m]","[clip - mlfoundations - ViT-B-32--laion400m_e31]","[clip - mlfoundations - ViT-B-32--laion400m_e32]","[clip - mlfoundations - ViT-B-32--laion400m_avg]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_e31]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_e32]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_avg]","[clip - mlfoundations - ViT-B-16--laion400m_e31]","[clip - mlfoundations - ViT-B-16--laion400m_e32]","[clip - sbert - ViT-B-32-multilingual-v1]","[clip - facebookresearch - clip_small_25ep]","[simclr - facebookresearch - simclr_small_25ep]","[slip - facebookresearch - slip_small_25ep]","[slip - facebookresearch - slip_small_50ep]","[slip - facebookresearch - slip_small_100ep]","[clip - facebookresearch - clip_base_25ep]","[simclr - facebookresearch - simclr_base_25ep]","[slip - facebookresearch - slip_base_25ep]","[slip - facebookresearch - slip_base_50ep]","[slip - facebookresearch - slip_base_100ep]","[clip - facebookresearch - clip_large_25ep]","[simclr - facebookresearch - simclr_large_25ep]","[slip - facebookresearch - slip_large_25ep]","[slip - facebookresearch - slip_large_50ep]","[slip - facebookresearch - slip_large_100ep]","[clip - facebookresearch - clip_base_cc3m_40ep]","[slip - facebookresearch - slip_base_cc3m_40ep]","[slip - facebookresearch - slip_base_cc12m_35ep]","[clip - facebookresearch - clip_base_cc12m_35ep]"] {allow-input: true}
model3 = "" # @param ["[clip - openai - RN50]","[clip - openai - RN101]","[clip - mlfoundations - RN50--yfcc15m]","[clip - mlfoundations - RN50--cc12m]","[clip - mlfoundations - RN50-quickgelu--yfcc15m]","[clip - mlfoundations - RN50-quickgelu--cc12m]","[clip - mlfoundations - RN101--yfcc15m]","[clip - mlfoundations - RN101-quickgelu--yfcc15m]","[clip - mlfoundations - ViT-B-32--laion400m_e31]","[clip - mlfoundations - ViT-B-32--laion400m_e32]","[clip - mlfoundations - ViT-B-32--laion400m_avg]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_e31]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_e32]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_avg]","[clip - mlfoundations - ViT-B-16--laion400m_e31]","[clip - mlfoundations - ViT-B-16--laion400m_e32]","[clip - sbert - ViT-B-32-multilingual-v1]","[clip - facebookresearch - clip_small_25ep]","[simclr - facebookresearch - simclr_small_25ep]","[slip - facebookresearch - slip_small_25ep]","[slip - facebookresearch - slip_small_50ep]","[slip - facebookresearch - slip_small_100ep]","[clip - facebookresearch - clip_base_25ep]","[simclr - facebookresearch - simclr_base_25ep]","[slip - facebookresearch - slip_base_25ep]","[slip - facebookresearch - slip_base_50ep]","[slip - facebookresearch - slip_base_100ep]","[clip - facebookresearch - clip_large_25ep]","[simclr - facebookresearch - simclr_large_25ep]","[slip - facebookresearch - slip_large_25ep]","[slip - facebookresearch - slip_large_50ep]","[slip - facebookresearch - slip_large_100ep]","[clip - facebookresearch - clip_base_cc3m_40ep]","[slip - facebookresearch - slip_base_cc3m_40ep]","[slip - facebookresearch - slip_base_cc12m_35ep]","[clip - facebookresearch - clip_base_cc12m_35ep]"] {allow-input: true}

if ViT_B32: 
  clip_load_list.append("[clip - mlfoundations - ViT-B-32--openai]")
if ViT_B16: 
  clip_load_list.append("[clip - mlfoundations - ViT-B-16--openai]")
if ViT_L14: 
  clip_load_list.append("[clip - mlfoundations - ViT-L-14--openai]")
if RN50x4: 
  clip_load_list.append("[clip - mlfoundations - RN50x4--openai]")
if RN50x64: 
  clip_load_list.append("[clip - mlfoundations - RN50x64--openai]")
if RN50x16: 
  clip_load_list.append("[clip - mlfoundations - RN50x16--openai]")
if ViT_L14_336px:
  clip_load_list.append("[clip - mlfoundations - ViT-L-14-336--openai]")
if ViT_B16_plus:
  clip_load_list.append("[clip - mlfoundations - ViT-B-16-plus-240--laion400m_e32]")
if ViT_B32_laion2b:
  clip_load_list.append("[clip - mlfoundations - ViT-B-32--laion2b_e16]")
if clip_farsi:
  clip_load_list.append("[clip - sajjjadayobi - clipfa]")
if clip_korean:
  clip_load_list.append("[clip - navervision - kelip_ViT-B/32]")
if cloob_ViT_B16:
  clip_load_list.append("[cloob - crowsonkb - cloob_laion_400m_vit_b_16_32_epochs]")

if model1:
  clip_load_list.append(model1)
if model2:
  clip_load_list.append(model2)
if model3:
  clip_load_list.append(model3)

majesty.clip_load_list = clip_load_list

torch.cuda.empty_cache()
gc.collect()

#### Advanced settings for the generation
##### Access [our guide](https://multimodal.art/majesty-diffusion)  

In [ ]:
opt = DotMap()

#Change it to false to not use CLIP Guidance at all 
majesty.use_cond_fn = True

#Custom cut schedules and super-resolution. Check out the guide on how to use it a https://multimodal.art/majestydiffusion
majesty.custom_schedule_setting = [
 [200,1000,8],
 [50,200,5],
 #"gfpgan:1.5",
 #[50,200,5],
]
              
#Cut settings
majesty.clamp_index = [1]*1000 
majesty.cut_overview = [8]*500 + [4]*500
majesty.cut_innercut = [0]*500 + [4]*500
majesty.cut_ic_pow = .1
majesty.cut_icgray_p = [.1]*300+[0]*1000
majesty.cutn_batches = 1
majesty.range_index = [0]*300 + [0]*1000 
majesty.active_function = "softsign" # function to manipulate the gradient - help things to stablize
majesty.tv_scales = [1000]*1+[600]*3
majesty.latent_tv_loss = True #Applies the TV Loss in the Latent space instead of pixel, improves generation quality

#If you uncomment next line you can schedule the CLIP guidance across the steps. Otherwise the clip_guidance_scale basic setting will be used
#majesty.clip_guidance_schedule = [10000]*300 + [500]*700

majesty.symmetric_loss_scale = 0 #Apply symmetric loss

#Latent Diffusion Advanced Settings
majesty.scale_div = 0.5 # Use when latent upscale to correct satuation problem
majesty.opt_mag_mul = 10 #Magnify grad before clamping
#PLMS Currently not working, working on a fix
#majesty.opt.plms = False #Won;=t work with clip guidance
majesty.opt_ddim_eta, opt_eta_end = [1.4,1] # linear variation of eta
majesty.opt_temperature = .975 

#Grad advanced settings
majesty.grad_center = False
majesty.grad_scale= 0.5 #5 Lower value result in more coherent and detailed result, higher value makes it focus on more dominent concept
majesty.anti_jpg = 0 #not working

#Init image advanced settings
majesty.init_rotate, majesty.mask_rotate=[False, False]
majesty.init_magnitude = 0.15

#More settings
majesty.RGB_min, majesty.RGB_max = [-0.95,0.95]
majesty.padargs = {"mode":"constant", "value": -1} #How to pad the image with cut_overview
majesty.flip_aug=False
majesty.cc = 60
majesty.cutout_debug = False
majesty.opt.outdir = outputs_path

#Experimental aesthetic embeddings, work only with OpenAI ViT-B/32 and ViT-L/14
majesty.experimental_aesthetic_embeddings = False
#How much you want this to influence your result
majesty.experimental_aesthetic_embeddings_weight = 0.5
#9 are good aesthetic embeddings, 0 are bad ones
majesty.experimental_aesthetic_embeddings_score = 9

### Prompts
The main prompt is the CLIP prompt. The Latent Prompts usually help with style and composition, you can turn them off by setting `latent_diffsion_guidance_scale=0` 

In [ ]:
#Amp up your prompt game with prompt engineering, check out this guide: https://matthewmcateer.me/blog/clip-prompt-engineering/
#Prompt for CLIP Guidance
clip_prompts = ["portrait of a princess in sanctuary, hyperrealistic painting trending on artstation"]

#Prompt for Latent Diffusion
latent_prompts = ["portrait of a princess in sanctuary, hyperrealistic painting trending on artstation"]

#Negative prompts for Latent Diffusion
latent_negatives = ["low quality image"]

majesty.image_prompts = []

### Diffuse!

In [ ]:
import warnings
warnings.filterwarnings('ignore')
#@markdown ### Basic settings 
#@markdown We're still figuring out default settings. Experiment and <a href="https://github.com/multimodalart/majesty-diffusion">share your settings with us</a>
majesty.width =  256#@param{type: 'integer'}
majesty.height =  256#@param{type: 'integer'}
majesty.latent_diffusion_guidance_scale = 2 #@param {type:"number"}
majesty.clip_guidance_scale = 5000 #@param{type: 'integer'}
majesty.how_many_batches = 1 #@param{type: 'integer'}
majesty.aesthetic_loss_scale = 200 #@param{type: 'integer'}
majesty.augment_cuts=True #@param{type:'boolean'}

#@markdown

#@markdown  ### Init image settings
#@markdown `init_image` requires the path of an image to use as init to the model
majesty.init_image = None #@param{type: 'string'}
if(majesty.init_image == '' or majesty.init_image == 'None'):
  majesty.init_image = None
#@markdown `starting_timestep`: How much noise do you want to add to your init image for it to then be difused by the model
majesty.starting_timestep = 0.9 #@param{type: 'number'}
#@markdown `init_mask` is a mask same width and height as the original image with the color black indicating where to inpaint
majesty.init_mask = None #@param{type: 'string'}
#@markdown `init_scale` controls how much the init image should influence the final result. Experiment with values around `1000`
majesty.init_scale = 1000 #@param{type: 'integer'}
majesty.init_brightness = 0.0 #@param{type: 'number'}
#@markdown How much extra noise to add to the init image, independently from skipping timesteps (use it also if you are upscaling)
majesty.init_noise = 0.6 #@param{type: 'number'}

#@markdown

#@markdown ### Custom saved settings
#@markdown If you choose custom saved settings, the settings set by the preset overrule some of your choices. You can still modify the settings not in the preset. <a href="https://github.com/multimodalart/majesty-diffusion/tree/main/latent_settings_library">Check what each preset modifies here</a>
majesty.custom_settings = 'path/to/settings.cfg' #@param{type:'string'}
settings_library = 'None (use settings defined above)' #@param ["None (use settings defined above)", "default (optimized for colab free)", "dango233_princesses", "the_other_zippy_defaults", "makeitrad_defaults"]
if(settings_library != 'None (use settings defined above)'):
  if(settings_library == 'default (optimized for colab free)'):
    majesty.custom_settings = f'majesty-diffusion/latent_settings_library/default.cfg'
  else:
    majesty.custom_settings = f'majesty-diffusion/latent_settings_library/{settings_library}.cfg'

majesty.load_custom_settings()

majesty.full_clip_load()

majesty.config_init_image()

majesty.prompts = clip_prompts
majesty.opt.prompt = latent_prompts
majesty.opt.uc = latent_negatives
majesty.set_custom_schedules()
majesty.config_clip_guidance()
majesty.config_output_size()
majesty.config_options()

torch.cuda.empty_cache()
gc.collect()
majesty.generate_video = False
majesty.do_run()

### Save your own settings


In [ ]:

#@markdown ### Save current settings
#@markdown If you would like to save your current settings, uncheck `skip_saving` and run this cell. You will get a `custom_settings.cfg` file you can reuse and share. If you like your results, send us a <a href="#">pull request</a> to add your settings to the selectable library
skip_saving = True #@param{type:'boolean'}
if(not skip_saving):
  data = majesty.generate_settings_file(add_prompts=False, add_dimensions=True)
  text_file = open("custom_settings.cfg", "w")
  text_file.write(data)
  text_file.close()
  from google.colab import files
  files.download('custom_settings.cfg')
  print("Downloaded as custom_settings.cfg")

### Biases acknowledgment
Despite how impressive being able to turn text into image is, beware to the fact that this model may output content that reinforces or exarcbates societal biases. According to the <a href='https://arxiv.org/abs/2112.10752' target='_blank'>Latent Diffusion paper</a>:<i> \"Deep learning modules tend to reproduce or exacerbate biases that are already present in the data\"</i>. 

The model was trained on an unfiltered version the LAION-400M dataset, which scrapped non-curated image-text-pairs from the internet (the exception being the the removal of illegal content) and is meant to be used for research purposes, such as this one. <a href='https://laion.ai/laion-400-open-dataset/' target='_blank'>You can read more on LAION's website</a>